# PyTorch: Using MLflow and Optuna for experiment tracking and hyperparameter optimization

## Imports

In [9]:
import optuna
import mlflow
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from mlflow import pytorch
from pprint import pformat

## Network 

In [10]:
class Net(nn.Module):
    def __init__(self, dropout=0.0):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

## Train and validation functions

In [11]:
# Training loop
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_set_size = len(train_loader.dataset)
    num_batches = len(train_loader)
    train_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            batch_size = len(data)
            print(f"Train Epoch: {epoch} [{batch_idx * batch_size}/{train_set_size} "
                  f"({100. * batch_idx / num_batches:.0f}%)]\tLoss: {loss.item():.6f}")
    avg_train_loss = train_loss / num_batches
    return avg_train_loss

# Testing loop
def validate(model, device, val_loader):
    model.eval()
    val_set_size = len(val_loader.dataset)
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= val_set_size

    print(f"Test set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{val_set_size} "
          f"({100. * correct / val_set_size:.0f}%)\n")
    return val_loss

## Main

In [12]:

# Obtain hyperparameters for this trial
def suggest_hyperparameters(trial):
    # Obtain the learning rate on a logarithmic scale
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    # Obtain the dropout ratio in a range from 0.0 to 0.9 with step size 0.1
    dropout = trial.suggest_float("dropout", 0.0, 0.9, step=0.1)
    # Obtain the optimizer to use by name
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "Adadelta"])

    print(f"Suggested hyperparameters: \n{pformat(trial.params)}")
    return lr, dropout, optimizer_name

In [13]:
def get_mnist_dataloaders(batch_size=8):
    # Load the MNIST train and test datasets and save them to ./data
    mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    train_loader = torch.utils.data.DataLoader(mnist_train,
                                                       batch_size=batch_size,
                                                       shuffle=True)
    mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    val_loader = torch.utils.data.DataLoader(mnist_test,
                                                      batch_size=1000,
                                                      shuffle=True)
    return train_loader, val_loader

In [14]:
def objective(trial):
    print("\n********************************\n")
    best_val_loss = float('Inf')
    
    # Start a new mlflow run
    with mlflow.start_run():
        # Get hyperparameter suggestions created by optuna and log them as params using mlflow
        lr, dropout, optimizer_name = suggest_hyperparameters(trial)
        mlflow.log_params(trial.params)

        # Use CUDA if GPU is available and log device as param using mlflow
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        mlflow.log_param("device", device)

        # Initialize network
        model = Net(dropout=dropout).to(device)

        # Pick an optimizer based on optuna's parameter suggestion
        if optimizer_name == "Adam":
            optimizer = optim.Adam(model.parameters(), lr=lr)
        if optimizer_name == "Adadelta":
            optimizer = optim.Adadelta(model.parameters(), lr=lr)
        scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
        
        # Get DataLoaders for MNIST train and validation set
        train_loader, val_loader = get_mnist_dataloaders()
        
        # Network training & validation loop
        for epoch in range(0, 5):
            avg_train_loss = train(model, device, train_loader, optimizer, epoch)
            avg_val_loss = validate(model, device, val_loader)
            
            if avg_val_loss <= best_val_loss:
                best_val_loss = avg_val_loss

            # Log average train and validation set loss metrics for the current epoch using mlflow
            mlflow.log_metric("avg_train_losses", avg_train_loss, step=epoch)
            mlflow.log_metric("avg_val_loss", avg_val_loss, step=epoch)
            
            scheduler.step()

        # pytorch.save_model(model, f"{artifact_path}/mnist_model")

    # Return the best validation loss achieved by the network.
    # This is needed as Optuna needs to know how the suggested hyperparameters are influencing the network loss.
    return best_val_loss

In [15]:
def main():
    # Create the optuna study which shares the experiment name
    study = optuna.create_study(study_name="pytorch-mlflow-optuna", direction="minimize")
    study.optimize(objective, n_trials=5)

    # Print optuna study statistics
    print("\n++++++++++++++++++++++++++++++++++\n")
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Trial number: ", trial.number)
    print("  Loss (trial value): ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [16]:
main()



********************************

Suggested hyperparameters: 
{'batch_size_power': 5,
 'dropout': 0.2,
 'lr': 0.00019429713691238943,
 'optimizer_name': 'Adadelta'}
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.304224
Train Epoch: 0 [80/60000 (0%)]	Loss: 2.327093
Train Epoch: 0 [160/60000 (0%)]	Loss: 2.304100
Train Epoch: 0 [240/60000 (0%)]	Loss: 2.328569
Train Epoch: 0 [320/60000 (1%)]	Loss: 2.272505
Train Epoch: 0 [400/60000 (1%)]	Loss: 2.273807
Train Epoch: 0 [480/60000 (1%)]	Loss: 2.251088
Train Epoch: 0 [560/60000 (1%)]	Loss: 2.327597
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.315940
Train Epoch: 0 [720/60000 (1%)]	Loss: 2.305280
Train Epoch: 0 [800/60000 (1%)]	Loss: 2.283312
Train Epoch: 0 [880/60000 (1%)]	Loss: 2.277167
Train Epoch: 0 [960/60000 (2%)]	Loss: 2.312655
Train Epoch: 0 [1040/60000 (2%)]	Loss: 2.315323
Train Epoch: 0 [1120/60000 (2%)]	Loss: 2.295681
Train Epoch: 0 [1200/60000 (2%)]	Loss: 2.286155
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.285789
Train Epoch: 0 [1360/60000 (2%)]	L

KeyboardInterrupt: 